In [56]:
!pip install openfisca-us

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [57]:
from openfisca_us import Microsimulation

sim = Microsimulation()

sim.calc("spm_unit_net_income")

          value       weight
0       28327.0   687.710022
1       76811.0  1516.949951
2       15350.0  1483.560059
3       54856.0   928.729980
4       27050.0  1467.910034
...         ...          ...
65615  152253.0   470.359985
65616   44606.0   312.350006
65617   17282.0   514.109985
65618   78821.0   516.250000
65619   53152.0   386.369995

[65620 rows x 2 columns]

In [68]:
from openfisca_us.api import *
from openfisca_us.model_api import *

BENEFITS = [
  "snap",
  "wic",
  "ssi",
]

class spm_unit_federal_tax(Variable):
    value_type = float
    entity = SPMUnit
    label = "Federal income tax"
    definition_period = YEAR
    unit = USD

    def formula(spm_unit, period, parameters):
        total_income = add(spm_unit, period, ["taxable_income"])
        return 0.4 * total_income

class flat_tax_reform(Reform):
  def apply(self):
     self.update_variable(spm_unit_federal_tax)
     self.neutralize_variable("ssi")
     #self.neutralize_variable("wic")
     #self.neutralize_variable("snap")
     #for removed in BENEFITS: 
        #self.neutralize_variable(removed)
     

# By default, OpenFisca-US uses reported net income in order to exactly match poverty rates.
# This reform defined on the next three lines disables that behaviour, by removing the reported 
# net income values. We apply this 'reform' to both the baseline and reformed simulations.

class turn_on_benefits_simulation(Reform):
  def apply(self):
    self.neutralize_variable("spm_unit_net_income_reported")

baseline = Microsimulation(turn_on_benefits_simulation)
reformed = Microsimulation((turn_on_benefits_simulation, flat_tax_reform))

/usr/local/lib/python3.7/dist-packages/openfisca_core/holders/holder.py:169: Warning: You cannot set a value for the variable spm_unit_net_income_reported, as it has been neutralized. The value you provided ([28327. 76811. 15350. ... 17282. 78821. 53152.]) will be ignored.
  Warning


In [69]:
# total funding raised: 2375.0273671257964
reformed.calc("spm_unit_net_income").sum() / 1e9 - baseline.calc("spm_unit_net_income").sum() / 1e9

-2146.056116657607

In [63]:
# total funding raised Flat Tax (no repeal of benefits): 2096.010503288724 
# additional revenue generated by neutralizing benefits: 2375.0273671257964 - 2096.010503288724 = 279.016864 

In [53]:
# total funding raised by the SNAP Abolition + Flat tax: 2327.931632698561
# additional revenue generated by neutralizing SNAP: 2327.931632698561 - 2096.010503288724 = 231.921129 


In [46]:
# total funding raised by the WIC Abolition + Flat tax: 2100.433048614732
# additional revenue generated by neutralizing WIC: 2100.433048614732 - 2096.010503288724 = 4.42254533

In [ ]:
# total funding raised by SSI Abolition + Flat tax: 2146.056116657607
# additioanl revenue generated by neutralizing SSI: 2146.056116657607 - 2096.010503288724 = 50.0456134